In [20]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#MongoDB Module
from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "pickle123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# grab all animal shelter related to dogs
df = pd.DataFrame.from_records(db.read({"animal_type": "Dog"}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id','datetime','monthyear','age'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


In [21]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Use div with display flex to align items horizontally and center
    html.Div([
        html.Img(src='assets/Grazioso Salvare Logo.png', style={'height':'100px', 'width':'100px'}), 
        html.H1('SNHU CS-340 Dashboard', style={'marginLeft': '20px'}),
    ], style={'display': 'flex', 'alignItems':'center', 'justifyContent':'center'}),
    html.Hr(),
#     html.Br(),
    html.Div([
        html.H1('Rescue Dog Type'),    
        dcc.RadioItems(
        id='data_filter', #id for callbacks
        options=[
        {'label': 'Water', 'value': 'Water'},
        {'label': 'Mountain or Wilderness','value': 'Mountain or Wilderness'},
        {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
        {'label': 'Reset', 'value': 'None'},
        ],
        value = 'None' ,
        inline=True
        )   
    ], style={'display': 'horizontal', 'alignItems':'left', 'justifyContent':'left'}),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        # Enabling single row selection
        row_selectable='single',

        # Enabling sorting
        sort_action='native',

        # Enabling filtering
        filter_action='native',

        # Enabling pagination
        page_action='native',
        page_current=0,  # page number that the user is currently on
        page_size=15,  # number of rows visible per page

        # Conditional formatting can be added here based on your requirements

        # Additional optional features for enhancing user interaction
        style_table={'overflowX': 'auto'},  # Horizontal scroll
        style_cell={
            'minWidth': '180px', 'width': '140px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },  # Ensuring text in each cell is visible without altering the table layout
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },  # Styling for the header to differentiate from row entries
),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            dcc.Graph(id='bar-chart'),
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    
#unique footer with my creator info
    html.Div([
            html.P("Creator:Nicholas Kreuziger", style={'margin-left':'10px', 'font-size':'16px'}),
            html.P([
                'Github Link: ',
                html.A("Syncert", href='https://github.com/Syncert', target="_blank")
            ], style={'margin-left': '10px', 'font-size': '16px'})
        ], style = {'text-align': 'left', 'margin-top':'10px'})
])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return [] #Return an empty list if none
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#This callback resets the selections on the data table to none when none is selected
@app.callback(
    Output('datatable-id', 'selected_rows'),  # Update the selected rows of the datatable
    [Input('data_filter', 'value')]  # Listen to changes in the value of the radio items
)
def reset_table_selection(selected_type):
    return [] #Always return a blank list so the selections reset every option selection


#This callback updates a pie chart of the animal types
# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('datatable-id', 'data')]
)
def update_breed_distribution_chart(data):
    if data is None or not data:
        return px.bar(title="No Data Available")
    
    df = pd.DataFrame(data)
    if 'breed' in df.columns:
        # Aggregate data to count the frequency of each breed
        breed_counts = df['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        
        # Create a horizontal bar chart
        fig = px.bar(breed_counts, x='count', y='breed', orientation='h', 
                     title="Distribution of Dog Breeds",
                     labels={'count':'Number of Dogs', 'breed':'Dog Breed'})  # You can rename the axes if you want
        fig.update_layout(
            autosize=False,
            width=1000,
            height=500,
            margin=dict(l=0, r=0, t=30, b=0),
            xaxis_title="Number of Dogs",
            yaxis_title=None  # Hides the axis title for breeds since it's evident from the data
        )
        return fig
    else:
        return px.bar(title="No Data Available for Breeds")



#This callback updates the data underlying the view based upon the animal type selections in the radio menu
@app.callback(
    Output('datatable-id', 'data'),  # Update the data of the datatable
    [Input('data_filter', 'value')]  # Listen to changes in the value of the radio items
)
def update_table(selected_type):
    if selected_type == 'None':
        query = {"animal_type": "Dog"}
        
    elif selected_type == 'Water':     
        query = {
            "animal_type": "Dog",
#IF DOG BREEDS MUST BE EXACTLY THE PREFERRED BREED
#            "breed": {"$in": ["Labrador Retriever Mix", \
#                    "Chesapeake Bay Retriever", \
#                    "Newfoundland"]}, \
#IF DOG BREEDS CAN BE WILDCARD SEARCHED
            "breed": {"$regex": "Labrador Retriever Mix|Chesapeake Bay Retriever|Newfoundland", "$options": "i"},
            "sex_upon_outcome":"Intact Female" ,
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}  # Correct range query
        }
        
    elif selected_type == 'Mountain or Wilderness':
        query = {
            "animal_type": "Dog",
#IF DOG BREEDS MUST BE EXACTLY THE PREFERRED BREED
#            "breed": {"$in": ["German Shepherd", \
#                                "Alaskan Malamute", \
#                                "Siberian Husky", \
#                               "Rottweiler"]}, \
#IF DOG BREEDS CAN BE WILDCARD SEARCHED
            "breed": {"$regex": "German Shepherd|Alaskan Malamute|Siberian Husky|Rottweiler","$options": "i"},
            "sex_upon_outcome":"Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}  # Correct range query 
        }
        
    elif selected_type == 'Disaster or Individual Tracking':
        query = {
            "animal_type": "Dog",
#IF DOG BREEDS MUST BE EXACTLY THE PREFERRED BREED
#            "breed": {"$in": ["Doberman Pinscher", \
#                                "German Shepherd", \
#                                "Golden Retriever", \
#                              "Bloodhound", \
#                               "Rottweiler"]}, \
#IF DOG BREEDS CAN BE WILDCARD SEARCHED
            "breed": {"$regex": "Doberman Pinscher|German Shepherd|Golden Retriever|Bloodhound|Rottweiler","$options": "i"},
            "sex_upon_outcome":"Intact Male" ,
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}  # Correct range query 
        }
    else:
        query = {"animal_type": "Dog"}

    # Fetch new data from MongoDB based on the selected type
    new_df = pd.DataFrame.from_records(db.read(query))
    new_df.drop(columns=['_id','datetime','monthyear'], inplace=True)  # Removing the '_id' column
    
    return new_df.to_dict('records')  # Update the table's data

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, selected_rows):
    #default center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        #Return default if no row is selected
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})]        


    # This conversion step is necessary to access the selected row correctly
    rows = pd.DataFrame.from_records(viewData)  # Convert viewData to a DataFrame
    
    # Now, you can access the selected_row as shown
    if selected_rows and viewData and selected_rows[0] < len(rows):
        
        selected_row = rows.iloc[selected_rows[0]] #if row is selected row
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        center = [latitude, longitude] #center on selected row
        zoom = 10 #Zoom in to location
        
        #create a marker
        marker = dl.Marker(position=center, children=[
            dl.Tooltip(selected_row['breed']),
            dl.Popup(html.Div([
                html.B("Animal Name:"),
                html.Br(),
                html.Div(selected_row['name']) #animal name
            ]))
        ])
        
        #Updating the map's children to include the new marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]

app.run_server(debug=True)

Dash app running on http://127.0.0.1:25127/
